In [1]:
%%info

In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1633282985681_0005,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{
    "conf":{
        "spark.jars.packages": "/usr/lib/spark/jars/delta-core_2.11-0.6.1",
        "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
        "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog"
    }
}

Starting Spark application


The code failed because of a fatal error:
	Session 5 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
Exception in thread "main" java.lang.IllegalArgumentException: requirement failed: Provided Maven Coordinates must be in the form 'groupId:artifactId:version'. The coordinate provided is: /usr/lib/spark/jars/delta-core_2.11-0.6.1
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.deploy.SparkSubmitUtils$$anonfun$extractMavenCoordinates$1.apply(SparkSubmit.scala:1022)
	at org.apache.spark.deploy.SparkSubmitUtils$$anonfun$extractMavenCoordinates$1.apply(SparkSubmit.scala:1020)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.IndexedSeqOptimized$class.foreach(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:186)
	at scala.collection.TraversableLike$class.map(Traversa

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,None,pyspark,dead,,,✔


In [65]:
df=spark.read.format("csv").option("header","true").option("inferSchema","true").load("s3://deltalake-emr/source-landing/products.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [66]:
spark = SparkSession \
        .builder \
        .appName("DeltaLake") \
        .config("spark.jars", "/usr/lib/spark/jars/delta-core_2.11-0.6.1")\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
spark.sparkContext.addPyFile("/usr/lib/spark/jars/delta-core_2.11-0.6.1.jar")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+-----+-------------------+-------------------+
|   product|category|price|          startdate|            enddata|
+----------+--------+-----+-------------------+-------------------+
|Samsung TX|  Tablet|  999|2020-01-02 01:00:00|2021-01-02 00:00:00|
|Samsung JX|  Mobile|  799|2020-01-03 02:00:00|2021-01-03 00:00:00|
|Redmi Note|  Mobile|  399|2020-01-04 03:00:00|2021-01-04 00:00:00|
|        Mi|  Mobile|  299|2020-01-05 04:00:00|2021-01-05 00:00:00|
|      iPad|  Tablet|  789|2020-01-06 05:00:00|2021-01-06 00:00:00|
|    iPhone|  Mobile|  999|2020-01-07 06:00:00|2021-01-07 00:00:00|
|  Micromax|  Mobile|  249|2020-01-08 07:00:00|2021-01-08 00:00:00|
|    Lenovo|  Tablet|  499|2020-01-09 08:00:00|2021-01-09 00:00:00|
|   OnePlus|  Mobile|  356|2020-01-10 09:00:00|2021-01-10 00:00:00|
|        Xu|  Tablet|  267|2020-01-11 10:00:00|2021-01-11 00:00:00|
+----------+--------+-----+-------------------+-------------------+

In [67]:
df.write.format("delta").mode("overwrite").save("s3://deltalake-emr/DeltaLake/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [68]:
delta_df=spark.read.format("delta").load("s3://deltalake-emr/DeltaLake/").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+-----+-------------+-------------+
|   product|category|price|    startdate|      enddate|
+----------+--------+-----+-------------+-------------+
|Samsung TX|  Tablet|  999|1/2/2020 1:00|1/2/2021 0:00|
|Samsung JX|  Mobile|  799|1/3/2020 2:00|1/3/2021 0:00|
|Redmi Note|  Mobile|  399|1/4/2020 3:00|1/4/2021 0:00|
|        Mi|  Mobile|  299|1/5/2020 4:00|1/5/2021 0:00|
|      iPad|  Tablet|  789|1/6/2020 5:00|1/6/2021 0:00|
+----------+--------+-----+-------------+-------------+
only showing top 5 rows

LET'S PERFORM SCD1. READ THE 1ST DELTA FILE 

In [69]:
source_scd1=spark.read.format("csv").option("header","true").option("inferSchema","true").load("s3://deltalake-emr/source-landing/product_ver1.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
source_scd1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+-----+
|   product|category|price|
+----------+--------+-----+
|Samsung TX|  Tablet| 1010|
|Redmi Note|  Mobile| 2000|
|        Mi|  Mobile| 3000|
|      iPad|  Tablet| 4000|
|    iPhone|  Mobile| 5000|
+----------+--------+-----+

LETS ADD TIMESTAMP TO THE DATAFRAME

In [138]:
from pyspark.sql.functions import *
scd1_df=source_scd1\
.withColumn("startdate",date_format(current_timestamp(),'yyyy-MM-dd HH:mm:ss'))\
.withColumn("enddate",to_timestamp(lit("9999-12-31 00:00:00"),'yyyy-MM-dd HH:mm:ss'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
scd1_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+-----+-------------------+-------------------+
|   product|category|price|          startdate|            enddate|
+----------+--------+-----+-------------------+-------------------+
|Samsung TX|  Tablet| 1010|2021-10-03 19:31:30|9999-12-31 00:00:00|
|Redmi Note|  Mobile| 2000|2021-10-03 19:31:30|9999-12-31 00:00:00|
|        Mi|  Mobile| 3000|2021-10-03 19:31:30|9999-12-31 00:00:00|
|      iPad|  Tablet| 4000|2021-10-03 19:31:30|9999-12-31 00:00:00|
|    iPhone|  Mobile| 5000|2021-10-03 19:31:30|9999-12-31 00:00:00|
+----------+--------+-----+-------------------+-------------------+

read DELTA table and update as SCD1(upsert)

In [73]:
from delta import *
from delta.tables import *
DELTA_TABLE= DeltaTable.forPath(spark, 's3://deltalake-emr/DeltaLake/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
DELTA_TABLE.alias("baseload").merge(
           scd1_df.alias("newdata"),
           "baseload.product=newdata.product"
           ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

let's verify if data has been updated or not

In [80]:
full_snapshot=spark.read.format("delta").load("s3://deltalake-emr/DeltaLake/")
spark.read.format("delta").load("s3://deltalake-emr/DeltaLake/").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+-----+-------------------+-------------------+
|   product|category|price|          startdate|            enddate|
+----------+--------+-----+-------------------+-------------------+
|Samsung TX|  Tablet| 1010|2021-10-03 19:31:42|9999-12-31 00:00:00|
|Redmi Note|  Mobile| 2000|2021-10-03 19:31:42|9999-12-31 00:00:00|
|      iPad|  Tablet| 4000|2021-10-03 19:31:42|9999-12-31 00:00:00|
|    iPhone|  Mobile| 5000|2021-10-03 19:31:42|9999-12-31 00:00:00|
|Samsung JX|  Mobile|  799|      1/3/2020 2:00|      1/3/2021 0:00|
|        Mi|  Mobile| 3000|2021-10-03 19:31:42|9999-12-31 00:00:00|
|  Micromax|  Mobile|  249|      1/8/2020 7:00|      1/8/2021 0:00|
|   OnePlus|  Mobile|  356|     1/10/2020 9:00|     1/10/2021 0:00|
|        Xu|  Tablet|  267|    1/11/2020 10:00|     1/11/2021 0:00|
|    Lenovo|  Tablet|  499|      1/9/2020 8:00|      1/9/2021 0:00|
+----------+--------+-----+-------------------+-------------------+

we can confirm that data is successfully updated. Now let's perform scd2.for that let's read another file and see the contents of that file.

In [129]:
source_scd2=spark.read.format("csv").option("header","true").option("inferSchema","true").load("s3://deltalake-emr/source-landing/product_ver2.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
source_scd2.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+-----+
|   product|category|price|
+----------+--------+-----+
|Samsung TX|  Tablet| 1010|
|      iPad|  Tablet| 4000|
|    iPhone|  Mobile| 5000|
+----------+--------+-----+

In [158]:
scd2_df=source_scd2\
.withColumn("startdate",date_format(current_timestamp(),'yyyy-MM-dd HH:mm:ss'))\
.withColumn("enddate",to_timestamp(lit("9999-12-31 00:00:00"),'yyyy-MM-dd HH:mm:ss'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [159]:
scd2_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+-----+-------------------+-------------------+
|   product|category|price|          startdate|            enddate|
+----------+--------+-----+-------------------+-------------------+
|Samsung TX|  Tablet| 1010|2021-10-03 20:51:02|9999-12-31 00:00:00|
|      iPad|  Tablet| 4000|2021-10-03 20:51:02|9999-12-31 00:00:00|
|    iPhone|  Mobile| 5000|2021-10-03 20:51:02|9999-12-31 00:00:00|
|      Moto|  Mobile| 7000|2021-10-03 20:51:02|9999-12-31 00:00:00|
|    lenovo|  Tablet| 9000|2021-10-03 20:51:02|9999-12-31 00:00:00|
+----------+--------+-----+-------------------+-------------------+

In [208]:
import pyspark.sql.types as types
                                            ##records that need to be closed
scd2_df.join(full_snapshot,"product","leftsemi")\
.withColumn("mergeKey",source_scd2.product)\
.union(                                    ##records that are existing but need to be inserted with new start date
   scd2_df.join(full_snapshot,"product","leftsemi")\
.selectExpr("*","NULL as mergeKey")\
).union (                                   ##record that are new 
scd2_df.join(full_snapshot,"product","leftanti").withColumn("mergeKey",lit(None).cast(types.StringType()))
).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+-----+-------------------+-------------------+----------+
|   product|category|price|          startdate|            enddate|  mergeKey|
+----------+--------+-----+-------------------+-------------------+----------+
|Samsung TX|  Tablet| 1010|2021-10-03 21:24:56|9999-12-31 00:00:00|Samsung TX|
|      iPad|  Tablet| 4000|2021-10-03 21:24:56|9999-12-31 00:00:00|      iPad|
|    iPhone|  Mobile| 5000|2021-10-03 21:24:56|9999-12-31 00:00:00|    iPhone|
|Samsung TX|  Tablet| 1010|2021-10-03 21:24:56|9999-12-31 00:00:00|      null|
|      iPad|  Tablet| 4000|2021-10-03 21:24:56|9999-12-31 00:00:00|      null|
|    iPhone|  Mobile| 5000|2021-10-03 21:24:56|9999-12-31 00:00:00|      null|
|      Moto|  Mobile| 7000|2021-10-03 21:24:56|9999-12-31 00:00:00|      null|
|    lenovo|  Tablet| 9000|2021-10-03 21:24:56|9999-12-31 00:00:00|      null|
+----------+--------+-----+-------------------+-------------------+----------+

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-----+-------------------+-------------------+
|product|category|price|          startdate|            enddate|
+-------+--------+-----+-------------------+-------------------+
|   Moto|  Mobile| 7000|2021-10-03 21:03:01|9999-12-31 00:00:00|
| lenovo|  Tablet| 9000|2021-10-03 21:03:01|9999-12-31 00:00:00|
+-------+--------+-----+-------------------+-------------------+